<a href="https://colab.research.google.com/github/noahdenicola/RL/blob/main/NoahDeNicola_TSP_InstaDeep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r ../requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python --version


Python 3.9.16
